Expected run time: 2 minutes

# Imports

In [1]:
from IPython.core.display import display, HTML 
display(HTML(
    "<style>.container {  !important;\
    } div.output_wrapper .output { padding-left: 14px; }</style>"
))

In [18]:
#alternatively, pip3 or other command to install for the appropriate version of python
#on your machine.
! pip install .

In [3]:
# you may need to restart your kernel before this step
import ensemble_predictor as ep
import numpy as np
import pandas as pd
import os

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load

Some example data has been included in this package to illustrate running the ensemble

In [4]:
#load cell characteristics to use in predicting
X = ep.read_hdf5('sample_features.hdf5')

In [5]:
#load post-CRISPR knockout viability estimates
Y = ep.read_hdf5("sample_achilles_effects.hdf5")

In [6]:
print(X.shape, Y.shape)

(693, 138) (693, 2)


In [7]:
print(Y[:5])

            BRAF (673)  ERBB2 (2064)
ACH-000004   -0.032777     -0.289101
ACH-000005   -0.005177     -0.379375
ACH-000007   -0.157759     -0.531896
ACH-000009   -0.170781     -1.103794
ACH-000011   -0.026527     -0.514542


# Running the full ensemble

We'll try just running the full ensemble first:

In [8]:
os.mkdir("test_results")
ep.run_subset(X=X, Y=Y, directory="test_results")

Using the following models:
['Histology', 'KitchenSink', 'Expression', 'Mutation', 'Fusion', 'GSEA', 'CN', 'Methylation', 'AllExpression', 'AllGenomics', 'Mutation+Exp', 'CN+Exp', 'Methylation+Exp']
aligning features
creating models
creating ensemble
fitting ensemble to 2 columns
25.421377 elapsed, 50% complete, 25.421377 estimated remaining
51.530469 elapsed, 100% complete, 0.000000 estimated remaining
saving results to test_results/save0_2


# Reviewing results

In [9]:
summary = pd.read_csv("test_results/save0_2_summary.csv")

In [10]:
# each fold's Pearson correlation score is given in the columns score0, score1...
summary['average_score'] = summary[[s for s in summary.columns
                                   if s.startswith('score')]
                                  ].median(axis=1)

In [11]:
# You should see with BRAF (673)'s top model with average
# score near 0.8. 
# The top scoring model for ERBB2 (2064) should be above 0.5
print(summary[['gene', 'model', 'average_score']].sort_values('average_score', ascending=False))

            gene            model  average_score
10    BRAF (673)     Mutation+Exp       0.804767
1     BRAF (673)      KitchenSink       0.776977
9     BRAF (673)      AllGenomics       0.760146
3     BRAF (673)         Mutation       0.758679
4     BRAF (673)           Fusion       0.728399
0     BRAF (673)        Histology       0.700874
11    BRAF (673)           CN+Exp       0.691363
5     BRAF (673)             GSEA       0.676606
7     BRAF (673)      Methylation       0.660698
8     BRAF (673)    AllExpression       0.649712
12    BRAF (673)  Methylation+Exp       0.644351
6     BRAF (673)               CN       0.612224
2     BRAF (673)       Expression       0.610994
14  ERBB2 (2064)      KitchenSink       0.515738
15  ERBB2 (2064)       Expression       0.506470
23  ERBB2 (2064)     Mutation+Exp       0.489823
24  ERBB2 (2064)           CN+Exp       0.470678
21  ERBB2 (2064)    AllExpression       0.462541
19  ERBB2 (2064)               CN       0.461909
22  ERBB2 (2064)    

These are the results found from averaging the correlations of out-of-sample predictions with the true gene effect across folds. We can load the actual predictions for a specific model, such as `"KitchenSink"`:

In [12]:
kitchen_sink = pd.read_csv('test_results/save0_2_KitchenSink_predictions.csv', index_col=0)

In [13]:
print(kitchen_sink.corrwith(Y))

BRAF (673)      0.767470
ERBB2 (2064)    0.473728
dtype: float64


# Training the ensemble for a specific subset of models, columns

We may only be interested in the results of particular group of models. Additionally, if we are running multiple ensembles in parallel, we may want to divide the problem by the columns in `Y` to be predicted. This will run the ensemble for only two models on only the last column. Note that the first part of the file names `"save1_2"` indicates that this includes columns 1-2 using python's indexing convention (starts at 0, second value not included).

In [14]:
ep.run_subset(X=X, Y=Y, start_col=1, n_col=1, included_models=['Expression', 'Mutation'],
              directory="test_results")

Using the following models:
['Expression', 'Mutation']
aligning features
creating models
creating ensemble
fitting ensemble to 1 columns
saving results to test_results/save1_2


# Building a custom model

A few classes are defined that inherit from sklearn's `RandomForestRegressor` and are useful for subsetting features. `KFilteredForest` filters features for the top `k` with highest correlation to the target. Here, we'll build a model that will use only the top 100 most correlated features.

In [15]:
model = ep.KFilteredForest(max_depth=4, n_estimators=100)

In [16]:
model.fit(X, Y['BRAF (673)'])

In [17]:
# You should see BRAF (673)_Hot with more than 0.7 feature importance
print(pd.Series(model.feature_importances_, index=model.feature_names
               ).sort_values(ascending=False)[:10])

BRAF (673)_Hot                  0.730074
BRAF (673)_CN                   0.102663
disease_sutype_melanoma_Cell    0.028314
BRAF (673)_Dam                  0.026816
BRAF (673)_Exp                  0.017251
ERBB2 (2064)_Exp                0.014911
ERBB2_4_MethCpG                 0.012854
ERBB2_3_MethCpG                 0.009346
ERBB2 (2064)_CN                 0.005279
disease_colorectal_Cell         0.004889
dtype: float64
